## Notebook to convert the 'average' input file to a new parameter file iteration

##### In this case to update the default pfile with the set of adjested parameter values from the averaged output of the SP calibraiton

In [3]:
# Find the parameters that are adjusted in the calibration process

In [4]:
import xarray as xr
import os
import netCDF4 as nc
import matplotlib.pyplot as plt
import shutil
import numpy as np

### Control vars

In [ ]:
tag_param_file='fates_params_vertsc_dayl.nc'
newpfile='fates_params_vertsc_dayl_SPcal_2.nc'
pfilesdir='../intermediate_pfiles/'
averagefile = 'average4.nc'

### Find the variables that are calibrated in the SP calibration procedure

##### Compare two files in the SP LhC output

In [13]:

directory_path = '..//lhc_param_files'

file='FATES_LH_347.nc'
file_path = os.path.join(directory_path, file)
data1 = xr.open_dataset(file_path)
file='FATES_LH_348.nc'
file_path = os.path.join(directory_path, file)
data2 = xr.open_dataset(file_path)

common_variables = set(data1.variables) & set(data2.variables)

# Iterate through the common variables and compare their values
parlist=[]
for var_name in common_variables:
    var_data1 = data1[var_name]
    var_data2 = data2[var_name]
    # Compare the values of the variables
    if not var_data1.equals(var_data2):
        
        print(f"Values of variable '{var_name}' are different")
        parlist.append(var_name)
print(parlist)

Values of variable 'fates_leaf_vcmaxha' are different
Values of variable 'fates_leaf_stomatal_intercept' are different
Values of variable 'fates_leaf_vcmax25top' are different
Values of variable 'fates_leaf_vcmaxhd' are different
Values of variable 'fates_leaf_stomatal_slope_ballberry' are different
Values of variable 'fates_leaf_jmaxse' are different
Values of variable 'fates_nonhydro_smpsc' are different
Values of variable 'fates_maintresp_leaf_atkin2017_baserate' are different
Values of variable 'fates_leaf_vcmaxse' are different
['fates_leaf_vcmaxha', 'fates_leaf_stomatal_intercept', 'fates_leaf_vcmax25top', 'fates_leaf_vcmaxhd', 'fates_leaf_stomatal_slope_ballberry', 'fates_leaf_jmaxse', 'fates_nonhydro_smpsc', 'fates_maintresp_leaf_atkin2017_baserate', 'fates_leaf_vcmaxse']


#### Create and read in the new base parameter file
- Use the default parameter file from the new FATES tag
- and make a new version of this, 
- and then copy the variables from the averaged file into that.
- remembering that the number of PFTs is different


In [29]:

shutil.copy(pfilesdir+tag_param_file, pfilesdir+newpfile)
basepfile = xr.open_dataset(pfilesdir+newpfilename)

#### Read in the SP calibrated average parameter file

In [25]:
averagepfile = xr.open_dataset(pfilesdir+averagefile)


#### Copy the value of the parameters in the average file into the new baseline file

In [78]:
newpfile=xr.open_dataset(pfilesdir+newpfilename)

for var_name in parlist:  
    print(var_name)
    base_var = newpfile[var_name].values
    average_var = averagepfile[var_name].values
    
    #print('av',average_var)
    #print('bs',base_var)
    if(newpfile[var_name].ndim==1):
        base_var[0:12] = average_var[0:12]  
    else:
        base_var[:,0:12] = average_var[:,0:12]    


newpfile.close()

fates_leaf_vcmaxha
fates_leaf_stomatal_intercept
fates_leaf_vcmax25top
fates_leaf_vcmaxhd
fates_leaf_stomatal_slope_ballberry
fates_leaf_jmaxse
fates_nonhydro_smpsc
fates_maintresp_leaf_atkin2017_baserate
fates_leaf_vcmaxse


In [84]:
for var_name in newpfile.variables:
    if var_name in averagepfile.variables:
        base_var = newpfile.variables[var_name]
        average_var = averagepfile.variables[var_name]
        #print(var_name,'ndim',average_var.ndim)
        if(base_var.ndim<2):
            base_var=base_var[:12]
            average_var=average_var[:12]
            if(np.dtype(base_var) =='float64'):
                diffs = abs(base_var-average_var)
            
                if (np.sum(diffs.values[:])>0.001):
                    print(f"Variable {var_name} is different in basepfile and averagepfile")
                    #print(average_var.values)
                    #print(base_var.values)
            else:
                print(f"Variable {var_name} is in basepfile but not in averagepfile")


Variable fates_allom_agb1 is different in basepfile and averagepfile
Variable fates_allom_agb2 is different in basepfile and averagepfile
Variable fates_allom_amode is different in basepfile and averagepfile
Variable fates_allom_blca_expnt_diff is different in basepfile and averagepfile
Variable fates_allom_d2bl1 is different in basepfile and averagepfile
Variable fates_allom_d2bl2 is different in basepfile and averagepfile
Variable fates_allom_d2ca_coefficient_max is different in basepfile and averagepfile
Variable fates_allom_d2ca_coefficient_min is different in basepfile and averagepfile
Variable fates_allom_d2h1 is different in basepfile and averagepfile
Variable fates_allom_d2h2 is different in basepfile and averagepfile
Variable fates_allom_d2h3 is different in basepfile and averagepfile
Variable fates_allom_dbh_maxheight is different in basepfile and averagepfile
Variable fates_allom_lmode is different in basepfile and averagepfile
Variable fates_cnp_eca_alpha_ptase is different

IndexError: too many indices

In [ ]:
print(basepfile['fates_leaf_vcmax25top'].values)